In [1]:
import os
import re
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Main folder path
IMAGES_FOLDER = r"C:\Users\Shreyash D. Shinde\imdb_wiki"

# Define subfolders to check
SUBFOLDERS = ["00", "01", "02"]

#  Lists to store data
image_paths = []
ages = []

# Loop through subfolders
for subfolder in SUBFOLDERS:
    folder_path = os.path.join(IMAGES_FOLDER, subfolder)

    if os.path.exists(folder_path):
        for file in os.listdir(folder_path):
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                full_path = os.path.join(folder_path, file)

                # Extract birth year & image year using regex
                match = re.search(r'_(\d{4})-\d+-\d+_(\d{4})', file)  # Extract YYYY-MM-DD & image year
                if match:
                    birth_year = int(match.group(1))  # Extract birth year
                    image_year = int(match.group(2))  # Extract image year
                    age = image_year - birth_year    # Calculate age
                    
                    if 0 < age < 100:  # Filter unrealistic ages
                        image_paths.append(full_path)
                        ages.append(age)

# Convert to NumPy array
image_paths = np.array(image_paths)
ages = np.array(ages, dtype=np.float32)

print(f" Total Images Processed: {len(image_paths)}")


✅ Total Images Processed: 14886


In [ ]:
# Split data into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(image_paths, ages, test_size=0.2, random_state=42)

print(f" Train Images: {len(X_train)}, Test Images: {len(X_test)}")


✅ Train Images: 11908, Test Images: 2978


In [ ]:
#  Image Preprocessing Function
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize
    return img

# Generator for Large Datasets
def image_data_generator(image_paths, ages, batch_size=32):
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_images = [load_and_preprocess_image(img) for img in image_paths[i:i+batch_size]]
            batch_ages = ages[i:i+batch_size]
            yield np.array(batch_images), np.array(batch_ages, dtype=np.float32)


In [ ]:
# Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(256, activation="relu"),
    Dense(1, activation="linear")  # Output age
])

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.001), loss="mean_squared_error", metrics=["mae"])

# Model Summary
model.summary()


c:\Users\Shreyash D. Shinde\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,516,289 (24.86 MB)

 Trainable params: 6,516,289 (24.86 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Set Batch Size
BATCH_SIZE = 32
EPOCHS = 10

# Create Data Generators
train_generator = image_data_generator(X_train, y_train, batch_size=BATCH_SIZE)
test_generator = image_data_generator(X_test, y_test, batch_size=BATCH_SIZE)

# Train Model
model.fit(train_generator, 
          steps_per_epoch=len(X_train) // BATCH_SIZE, 
          epochs=EPOCHS, 
          validation_data=test_generator, 
          validation_steps=len(X_test) // BATCH_SIZE)

# Save Model
model.save("age_detection_model.h5")
print(" Model Saved Successfully!")


Epoch 1/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 191s 504ms/step - loss: 266.7590 - mae: 12.6850 - val_loss: 167.0864 - val_mae: 10.4997
Epoch 2/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 182s 490ms/step - loss: 181.9109 - mae: 10.7216 - val_loss: 166.4901 - val_mae: 10.5020
Epoch 3/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 182s 489ms/step - loss: 177.6710 - mae: 10.5824 - val_loss: 161.8023 - val_mae: 10.2435
Epoch 4/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 182s 488ms/step - loss: 174.4461 - mae: 10.4859 - val_loss: 159.7610 - val_mae: 10.1092
Epoch 5/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 181s 488ms/step - loss: 172.3891 - mae: 10.4214 - val_loss: 160.5275 - val_mae: 10.1106
Epoch 6/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 182s 489ms/step - loss: 170.4883 - mae: 10.3731 - val_loss: 163.1418 - val_mae: 10.0866
Epoch 7/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 182s 488ms/step - loss: 165.3148 - mae: 10.1909 - val_loss: 163.2819 - val_mae: 10.0674
Epoch 8/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 181s 487ms/step - loss: 160.8828 - mae: 10.0322 - val_loss: 165.

✅ Model Saved Successfully!


In [9]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

# Load the trained model
model = tf.keras.models.load_model("age_detection_model.h5")
print(" Model Loaded Successfully!")

# Function to preprocess the image
def preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    
    if img is None:
        print(f" Error: Could not load image {image_path}")
        return None
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to predict age
def predict_age(image_path):
    input_image = preprocess_image(image_path)
    
    if input_image is not None:
        predicted_age = model.predict(input_image)[0][0]
        result_label.config(text=f"🎯 Predicted Age: {predicted_age:.2f} years")

# Function to open file dialog
def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    
    if file_path:
        img = Image.open(file_path)
        img.thumbnail((250, 250))  # Resize for display
        img = ImageTk.PhotoImage(img)

        # Update image display
        image_label.config(image=img)
        image_label.image = img

        # Predict age
        predict_age(file_path)

# Create GUI window
root = tk.Tk()
root.title("Age Detection")

# Add a button to select an image
select_button = tk.Button(root, text=" Select Image", command=select_image)
select_button.pack(pady=10)

# Label to display selected image
image_label = tk.Label(root)
image_label.pack()

# Label to show prediction result
result_label = tk.Label(root, text=" Predicted Age: --", font=("Arial", 14))
result_label.pack(pady=10)

# Run GUI
root.mainloop()


 Model Loaded Successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
